## 문제 2-3 : 학생 정보 구조화 시스템
### 출력 파서: PydanticOutputParser

### 문제
#### 학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.

### 요구사항:
#### PydanticOutputParser와 BaseModel 사용
#### 각 필드에 적절한 타입과 설명 추가
#### 자유 형식의 텍스트에서 정보 추출

### 예시 입력:
#### "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

### 예시 출력 구조:
#### {
####    "name": "김민수",
####    "age": 22,
####    "major": "컴퓨터공학",
####    "hobbies": ["게임하기", "영화보기", "코딩"],
####    "goal": "훌륭한 개발자가 되는 것"
#### }

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='../.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:7])

gsk_t3Z


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

In [10]:
# 출력 구조를 정의하는 Pydantic 모델
class StudentProfile(BaseModel):
    name: str = Field(..., description="학생의 이름(풀네임)")
    age: int = Field(..., description="나이(정수)")
    major: str = Field(..., description="전공(예: 컴퓨터공학)")
    hobbies: List[str] = Field(..., description="취미 리스트(예: ['게임하기','영화보기'])")
    goal: str = Field(..., description="학생의 핵심 목표 한 문장")

In [21]:
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentProfile)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "학생의 이름(풀네임)", "title": "Name", "type": "string"}, "age": {"description": "나이(정수)", "title": "Age", "type": "integer"}, "major": {"description": "전공(예: 컴퓨터공학)", "title": "Major", "type": "string"}, "hobbies": {"description": "취미 리스트(예: ['게임하기','영화보기'])", "items": {"type": "string"}, "title": "Hobbies", "type": "array"}, "goal": {"description": "학생의 핵심 목표 한 문장", "title": "Goal", "type": "string"}}, "required": ["name", "age", "major", "hobbies", "goal"]}
```


In [23]:
template = """
    ("system",
     "너는 정보 추출기야. 사용자가 입력한 자기소개 텍스트에서 "
     "이름(name), 나이(age), 전공(major), 취미 목록(hobbies), 목표(goal)를 추출할것.\n"
     "반드시 아래 출력 형식을 따라야돼.\n"
     f"{format_instructions}\n"
     "- 취미는 리스트(List[str])로, 불필요한 설명 없이 항목만 포함하고\n"
     "- 값이 확실하지 않더라도 문맥상 가장 그럴듯한 값 한 개로 정제해."),
    ("user", "{query}")
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

In [24]:
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="openai/gpt-oss-120b",
    temperature=0.1,
)

In [26]:
# 체인 구성 및 실행
chain = prompt | llm | parser

query = (
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. "
    "취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
)

output = chain.invoke({"query": query})

print(type(output))
pprint(output)

<class '__main__.StudentProfile'>
StudentProfile(name='김민수', age=22, major='컴퓨터공학', hobbies=['게임하기', '영화보기', '코딩'], goal='훌륭한 개발자가 되는 것')


In [32]:
# 결과 출력
print(f"이름: {output.name}")
print(f"나이: {output.age}")
print(f"전공: {output.major}")
print(f"취미: {output.hobbies}")
print(f"목표: {output.goal}")

이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: ['게임하기', '영화보기', '코딩']
목표: 훌륭한 개발자가 되는 것
